# MODNet on the joined PBE and experimental datasets

Here we joined the PBE and experimental datasets, and trained the MODNet model on it. We weighted the data from the experimental dataset a bit more such that: {'pbe_gap':0.3, 'exp_gap':0.7}.

In [1]:
from modnet.preprocessing import MODData
from modnet.models import MODNetModel
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

In [2]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [3]:
md_exp = MODData.load('exp_gap_all_mpid')
md_exp.df_targets.columns = ['exp_gap','mp_id']
md_pbe = MODData.load('pbe_gap.zip')
#md_pbe.df_targets.columns = ['gap']
md_hse = MODData.load('hse_gap.zip')
#md_hse.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5e64745640> object, created with modnet version <=0.1.7
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5ddc57d910> object, created with modnet version <=0.1.7
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5ddc57dc40> object, created with modnet version 0.1.8~develop
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5ddc57d8b0> object, created with modnet version <=0.1.7


In [ ]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    '''fpath = 'trainA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
    '''   
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model = MODNetModel([[['pbe_gap'],['exp_gap']]],{'pbe_gap':0.5, 'exp_gap':0.5})
    model.fit_preset(train,verbose=0)
    
    pred = model.predict(test).drop(columns='pbe_gap')
    true = test.df_targets.drop(columns='pbe_gap')
    error = pred-true
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

In [8]:
maes

array([0.45579268, 0.4119005 , 0.35093791, 0.39612838, 0.45900535])

In [9]:
maes.mean()

0.41475296417363144

In [10]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'trainA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model = MODNetModel([[['pbe_gap'],['exp_gap']]],{'pbe_gap':0.4, 'exp_gap':0.6})
    model.fit_preset(train,verbose=0)
    
    pred = model.predict(test).drop(columns='pbe_gap')
    true = test.df_targets.drop(columns='pbe_gap')
    error = pred-true
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f577007a970> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.291
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.323
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.310
IN

0.45689708011281627


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.289
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.295
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.289
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.288
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.269
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.277
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.242
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.255
INFO:root:Training preset #9/16


0.4953519127419204


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.299
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.291
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.274
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.279
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.252
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.252
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.260
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.262
INFO:root:Training preset #9/16


0.3313009105387375


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.328
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.361
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.336
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.360
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.336
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.322
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.330
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.330
INFO:root:Training preset #9/16


0.3651834809757097


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.341
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.291
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.337
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.313
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.300
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.307
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.315
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.304
INFO:root:Training preset #9/16


0.4359251191643285


In [11]:
maes

array([0.45689708, 0.49535191, 0.33130091, 0.36518348, 0.43592512])

In [12]:
maes.mean()

0.4169317007067025

In [4]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    '''fpath = 'trainA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
    '''     
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model = MODNetModel([[['pbe_gap'],['exp_gap']]],{'pbe_gap':0.3, 'exp_gap':0.7})
    model.fit_preset(train,verbose=0)
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loading cross NMI from 'Features_cross' file.
INFO:root:Starting target 1/2: exp_gap ...
INFO:root:Computing mutual information between features and target...
INFO:root:Computing optimal features...
INFO:root:Selected 50/1102 features...
INFO:root:Selected 100/1102 features...
INFO:root:Selected 150/1102 features...
INFO:root:Selected 200/1102 features...
INFO:root:Selected 250/1102 features...
INFO:root:Selected 300/1102 features...
INFO:root:Selected 350/1102 feat

0.3544316232036284


INFO:root:Computing optimal features...
INFO:root:Selected 50/1095 features...
INFO:root:Selected 100/1095 features...
INFO:root:Selected 150/1095 features...
INFO:root:Selected 200/1095 features...
INFO:root:Selected 250/1095 features...
INFO:root:Selected 300/1095 features...
INFO:root:Selected 350/1095 features...
INFO:root:Selected 400/1095 features...
INFO:root:Selected 450/1095 features...
INFO:root:Selected 500/1095 features...
INFO:root:Selected 550/1095 features...
INFO:root:Selected 600/1095 features...
INFO:root:Selected 650/1095 features...
INFO:root:Selected 700/1095 features...
INFO:root:Selected 750/1095 features...
INFO:root:Selected 800/1095 features...
INFO:root:Selected 850/1095 features...
INFO:root:Selected 900/1095 features...
INFO:root:Selected 950/1095 features...
INFO:root:Selected 1000/1095 features...
INFO:root:Selected 1050/1095 features...
INFO:root:Done with target 1/2: exp_gap.
INFO:root:Starting target 2/2: pbe_gap ...
INFO:root:Computing mutual informat

0.3199206514540527


INFO:root:Computing optimal features...
INFO:root:Selected 50/1104 features...
INFO:root:Selected 100/1104 features...
INFO:root:Selected 150/1104 features...
INFO:root:Selected 200/1104 features...
INFO:root:Selected 250/1104 features...
INFO:root:Selected 300/1104 features...
INFO:root:Selected 350/1104 features...
INFO:root:Selected 400/1104 features...
INFO:root:Selected 450/1104 features...
INFO:root:Selected 500/1104 features...
INFO:root:Selected 550/1104 features...
INFO:root:Selected 600/1104 features...
INFO:root:Selected 650/1104 features...
INFO:root:Selected 700/1104 features...
INFO:root:Selected 750/1104 features...
INFO:root:Selected 800/1104 features...
INFO:root:Selected 850/1104 features...
INFO:root:Selected 900/1104 features...
INFO:root:Selected 950/1104 features...
INFO:root:Selected 1000/1104 features...
INFO:root:Selected 1050/1104 features...
INFO:root:Selected 1100/1104 features...
INFO:root:Done with target 1/2: exp_gap.
INFO:root:Starting target 2/2: pbe_ga

0.29445660018455794


INFO:root:Computing optimal features...
INFO:root:Selected 50/1099 features...
INFO:root:Selected 100/1099 features...
INFO:root:Selected 150/1099 features...
INFO:root:Selected 200/1099 features...
INFO:root:Selected 250/1099 features...
INFO:root:Selected 300/1099 features...
INFO:root:Selected 350/1099 features...
INFO:root:Selected 400/1099 features...
INFO:root:Selected 450/1099 features...
INFO:root:Selected 500/1099 features...
INFO:root:Selected 550/1099 features...
INFO:root:Selected 600/1099 features...
INFO:root:Selected 650/1099 features...
INFO:root:Selected 700/1099 features...
INFO:root:Selected 750/1099 features...
INFO:root:Selected 800/1099 features...
INFO:root:Selected 850/1099 features...
INFO:root:Selected 900/1099 features...
INFO:root:Selected 950/1099 features...
INFO:root:Selected 1000/1099 features...
INFO:root:Selected 1050/1099 features...
INFO:root:Done with target 1/2: exp_gap.
INFO:root:Starting target 2/2: pbe_gap ...
INFO:root:Computing mutual informat

0.3234598909779669


INFO:root:Computing optimal features...
INFO:root:Selected 50/1094 features...
INFO:root:Selected 100/1094 features...
INFO:root:Selected 150/1094 features...
INFO:root:Selected 200/1094 features...
INFO:root:Selected 250/1094 features...
INFO:root:Selected 300/1094 features...
INFO:root:Selected 350/1094 features...
INFO:root:Selected 400/1094 features...
INFO:root:Selected 450/1094 features...
INFO:root:Selected 500/1094 features...
INFO:root:Selected 550/1094 features...
INFO:root:Selected 600/1094 features...
INFO:root:Selected 650/1094 features...
INFO:root:Selected 700/1094 features...
INFO:root:Selected 750/1094 features...
INFO:root:Selected 800/1094 features...
INFO:root:Selected 850/1094 features...
INFO:root:Selected 900/1094 features...
INFO:root:Selected 950/1094 features...
INFO:root:Selected 1000/1094 features...
INFO:root:Selected 1050/1094 features...
INFO:root:Done with target 1/2: exp_gap.
INFO:root:Starting target 2/2: pbe_gap ...
INFO:root:Computing mutual informat

0.3696453115442744


In [5]:
maes

array([0.35443162, 0.31992065, 0.2944566 , 0.32345989, 0.36964531])

In [6]:
maes.mean()

0.33238281547289605

In [19]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'trainA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model = MODNetModel([[['pbe_gap'],['exp_gap']]],{'pbe_gap':0.2, 'exp_gap':0.8})
    model.fit_preset(train,verbose=0)
    
    pred = model.predict(test).drop(columns='pbe_gap')
    true = test.df_targets.drop(columns='pbe_gap')
    error = pred-true
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f27f6709280> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.371
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.358
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.320
IN

0.42581800814314263


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.322
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.282
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.278
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.298
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.272
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.306
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.268
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.242
INFO:root:Training preset #9/16


0.42444648365274623


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.299
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.309
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.317
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.309
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.289
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.340
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.290
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.301
INFO:root:Training preset #9/16


0.30619532573835767


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f26c0a85430> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.401
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.339
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.354
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.347
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.353
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.351
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.379
INFO:root:Training preset #8/16
INFO:r

0.3725547441065846


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.350
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.365
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.338
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.346
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.341
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.340
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.361
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.351
INFO:root:Training preset #9/16


0.3926373649157843


In [20]:
maes

array([0.42581801, 0.42444648, 0.30619533, 0.37255474, 0.39263736])

In [21]:
maes.mean()

0.3843303853113231

In [4]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'trainA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model = MODNetModel([[['pbe_gap'],['exp_gap']]],{'pbe_gap':0.1, 'exp_gap':0.9})
    model.fit_preset(train,verbose=0)
    
    pred = model.predict(test).drop(columns='pbe_gap')
    true = test.df_targets.drop(columns='pbe_gap')
    error = pred-true
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5918a7af40> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.355
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.328
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.302
IN

0.46757137030218177


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5918a7af40> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.299
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.299
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.302
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.285
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.279
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.315
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.262
INFO:root:Training preset #8/16
INFO:r

0.3867474432703408


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f587d28a3a0> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.334
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.299
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.307
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.306
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.281
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.309
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.299
INFO:root:Training preset #8/16
INFO:r

0.36591574097811497


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f587df4ba00> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.389
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.406
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.398
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.373
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.366
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.368
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.375
INFO:root:Training preset #8/16
INFO:r

0.4523466939934473


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f576409e040> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.376
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.372
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.406
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.341
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.349
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.350
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.354
INFO:root:Training preset #8/16
INFO:r

0.41085058988188244


In [5]:
maes

array([0.46757137, 0.38674744, 0.36591574, 0.45234669, 0.41085059])

In [6]:
maes.mean()

0.41668636768519346

In [4]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'trainA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model = MODNetModel([[['pbe_gap'],['exp_gap']]],{'pbe_gap':0.0, 'exp_gap':1.0})
    model.fit_preset(train,verbose=0)
    
    pred = model.predict(test).drop(columns='pbe_gap')
    true = test.df_targets.drop(columns='pbe_gap')
    error = pred-true
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f2ab4603880> object, created with modnet version 0.1.8
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.343
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.357
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.355
IN

0.4136553812497131


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.320
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.295
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.301
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.306
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.304
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.323
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.293
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.272
INFO:root:Training preset #9/16


0.39285292584804565


INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.317
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.330
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.309
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.312
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.332
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.301
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.338
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.347
INFO:root:Training preset #9/16
INFO:root:Compiling model...
INF

0.3274278537420169


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.397
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.393
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.424
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.397
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.391
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.375
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.405
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.371
INFO:root:Training preset #9/16


0.39402820611875833


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.414
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.403
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.392
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.434
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.345
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.365
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.408
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.357
INFO:root:Training preset #9/16


0.3726229517710718


In [5]:
maes

array([0.41365538, 0.39285293, 0.32742785, 0.39402821, 0.37262295])

In [6]:
maes.mean()

0.38011746374592115

#### Conclusion

For this dataset, the lower MAE is obtained for {'pbe_gap':0.3, 'exp_gap':0.7}, and the improvement is of about 9%.